# Dataset Caro y Cuervo

En este notebook se va a limpiar y preprocesar el resultado en markdown de la herramienta Marker para consolidar el dataset de BDC

In [5]:
import re
import unicodedata
import pandas as pd
import json

path_source = "Diccionario_breve_de_Colombiaismos_slown_text.txt"

# ============================================================================
# FUNCIONES BÁSICAS DE NORMALIZACIÓN Y LIMPIEZA DE TEXTO
# ============================================================================

def nfc(s: str) -> str:
    """Normaliza texto a forma canónica de composición Unicode (NFC)."""
    return unicodedata.normalize("NFC", s or "")

def norm_spaces(s: str) -> str:
    """Reemplaza espacios no separables y de ancho cero, y normaliza múltiples espacios."""
    s = (s or "").replace("\u00A0", " ").replace("\u200b", "")
    s = re.sub(r"[ \t]+", " ", s)
    return s.strip()

st_end_punt_re = re.compile(r"^[¡!¿?«»\"'()\[\]{}·•.,;:—–-]+|[¡!¿?«»\"'()\[\]{}·•.,;:—–-]+$")
def strip_punct(s: str) -> str:
    """
    Elimina signos de puntuación ubicados al inicio o al final de la cadena.
    No afecta la puntuación que está en el medio.

    Ejemplo:

    Entrada: "**¡ahijuelita!**"
    Salida:  "ahijuelita"
    """
    return st_end_punt_re.sub("", s or "").strip()


def clean_curs(s: str) -> str:
    """
    Quita el marcado de negrita y cursiva en Markdown, dejando solo el texto limpio.

    - Elimina `**` (negrita)
    - Elimina `__` (subrayado)
    - Reemplaza `*palabra*` por `palabra` (estaba en cursiva)

    Ejemplo:

    Entrada: "**¡ahijuelita!**  *bonita*"
    Salida:  "¡ahijuelita!  bonita"
    """
    s = (s or "").replace("**", "").replace("__", "")
    s = re.sub(r"\*(.*?)\*", r"\1", s)
    return s

remove_trailing_pipes_pattern = re.compile(r"\s*\|+\s*$")

def remove_trailing_pipes(s: str) -> str:
    """
    Elimina las barras verticales '|' que aparecen al final de la cadena.
    También remueve los espacios que puedan estar antes de las barras.

    Ejemplo:

    Entrada: "ser vivo ||"
    Salida:  "ser vivo"

    Entrada: "abeja |"
    Salida:  "abeja"
    """
    return remove_trailing_pipes_pattern.sub("", s or "").strip()

remove_gender_suffix_pattern = re.compile(r"\s*,\s*(?:a|da)\b\.?\s*$", re.IGNORECASE)

def remove_gender_suffix(token: str) -> str:
    """
    Quita el sufijo literal ", a" o ", da" al final de un lema para quedarse con la forma base.
    Si no encuentra el sufijo, deja el texto sin cambios.

    Ejemplo:

    Entrada: "bacano, a"
    Salida:  "bacano"

    Entrada: "cansado, da"
    Salida:  "cansado"

    Entrada: "árbol"
    Salida:  "árbol"
    """
    before = token
    after = remove_gender_suffix_pattern.sub("", token)
    return (after.strip() or before.strip())

# ============================================================================
# EXPANSIÓN DE TILDES (~) EN FRASES DERIVADAS
# ============================================================================

def replace_tilde_with_base(text: str, base: str) -> str:
    """
    Reemplaza la tilde "~" en una frase derivada por la palabra base del lema.
    Mantiene correctamente sufijos, espacios y signos de puntuación.

    Ejemplos:

    - Entrada: texto="~ de encostalados", palabra_base="carrera"
      Salida:  "carrera de encostalados"

    - Entrada: texto="echar ~", palabra_base="carreta"
      Salida:  "echar carreta"

    - Entrada: texto="hablar ~.", palabra_base="carreta"
      Salida:  "hablar carreta."

    - Entrada: texto="~osa", palabra_base="carranch"
      Salida:  "carranchosa"
    """
    if not text:
        return text
    # ~ seguida de letras (sufijos)
    text = re.sub(r"~([a-záéíóúüñ]+)", lambda m: base + m.group(1), text, flags=re.IGNORECASE)
    # ~ seguida de espacios y letras
    text = re.sub(r"~(\s+)([a-záéíóúüñ]+)", lambda m: base + m.group(1) + m.group(2), text, flags=re.IGNORECASE)
    # ~ seguida de puntuación
    text = re.sub(r"~([.,;:?!])", lambda m: base + m.group(1), text)
    # ~ restantes
    text = text.replace("~", base)
    return text

# ============================================================================
# PROCESAMIENTO DE LEMAS CON PIPES (||) Y TILDE (~)
# ============================================================================

def resolve_pipe_tilde_in_head(head_bold_content: str) -> str:
    """
    Resuelve el caso de encabezados que usan "||" (pipes) y tilde "~".
    Toma la parte DERECHA del "||" y reemplaza "~" por la palabra base que está a la izquierda.

    Ejemplos:

    - Entrada: lema_con_barras = "carrera || ~ de encostalados"
      Salida:  "carrera de encostalados"

    - Entrada: lema_con_barras = "carraca || echar ~"
      Salida:  "echar carraca"

    - Entrada: lema_con_barras = "carreta || echar o hablar ~"
      Salida:  "echar o hablar carreta"
    """
    parts = head_bold_content.split("||", 1)
    if len(parts) != 2:
        return None
    left, right = parts[0], parts[1]
    base = strip_punct(clean_curs(norm_spaces(left)))
    base = remove_gender_suffix(base)
    base = strip_punct(base)
    rhs = norm_spaces(right)
    rhs = replace_tilde_with_base(rhs, base)
    rhs = strip_punct(rhs)
    rhs = remove_trailing_pipes(rhs)
    rhs = norm_spaces(rhs)
    return rhs

# ============================================================================
# EXTRACCIÓN Y NORMALIZACIÓN DE LEMAS
# ============================================================================

re_bold_head = re.compile(r"^-?\s*\*\*(.*?)\*\*")

def extract_bold_head_general(line: str) -> str:
    """
    Extrae el lema (palabra principal) que aparece encerrado entre **negritas** 
    al inicio de una línea en el diccionario.

    Ejemplos:

    - Entrada: linea = "- **carraca** f. coloq. Mandíbula del hombre o los animales."
      Salida:  "carraca"

    - Entrada: linea = "- **carreta || echar o hablar ~** fr. coloq. Hablar cosas triviales..."
      Salida:  "carreta || echar o hablar ~"

    - Entrada: linea = "- **carranchil (carranchín)** m. Enfermedad cutánea caracterizada por un fuerte escozor."
      Salida:  "carranchil (carranchín)"

    - Entrada: linea = "- **carrera || ~ de encostalados** Competencia deportiva en la cual los participantes..."
      Salida:  "carrera || ~ de encostalados"
    """
    m = re_bold_head.search(line)
    return m.group(1) if m else None

def remove_optional_plural_suffix(token: str) -> str:
    """
    Elimina sufijos de plural opcional "(s)" o "(es)" de un lema.

    Esto es común en el diccionario cuando un lema admite plural, 
    pero no se quiere que aparezca como parte de la palabra base.

    Ejemplos :
    
    - Entrada: "botarata(s)"  
      Salida:  "botarata"

    - Entrada: "carriel(es)"  
      Salida:  "carriel"

    - Entrada: "mataburro(s)"  
      Salida:  "mataburro"
    """
    return re.sub(r"\((?:s|es)\)$", "", token, flags=re.IGNORECASE)

def normalize_headword_content(head_content: str) -> str:
    """
    Normaliza el lema extraído de la cabecera en **negrita** de un artículo.

    Acciones realizadas:
    1. Normaliza caracteres Unicode a forma NFC.
    2. Si el lema contiene '||' (derivados con "~"), los resuelve
       reemplazando "~" por la palabra base.
    3. Elimina sufijos de plural opcional "(s)" o "(es)".
    4. Descarta variantes indicadas entre paréntesis después del lema.
    5. Elimina colas literales como ", a" o ", da" (marcas de género).
    6. Limpia signos de puntuación y barras verticales sobrantes.
    7. Normaliza espacios en blanco.

    Ejemplos :
    
    - Entrada: "carranchil (carranchín)"
      Salida:  "carranchil"

    - Entrada: "carrera || ~ de encostalados"
      Salida:  "carrera de encostalados"

    - Entrada: "botarata(s)"
      Salida:  "botarata"
    """
    head = nfc(head_content)
    if "||" in head:
        resolved = resolve_pipe_tilde_in_head(head)
        if resolved:
            return nfc(remove_optional_plural_suffix(resolved))
    token = norm_spaces(head)
    token = remove_optional_plural_suffix(token)
    token = token.split(" (", 1)[0]
    token = remove_gender_suffix(token)
    token = strip_punct(token)
    token = remove_trailing_pipes(token)
    token = norm_spaces(token)
    return nfc(token)

# ============================================================================
# PATRONES Y CONFIGURACIÓN DE LIMPIEZA DE METADATOS
# ============================================================================

# Detecta ocurrencias externas: "|| **frase_con_tilde** resto_del_texto"
external_occurrence_pattern = re.compile(
    r"\|\|\s*\*{2}\s*(.*?)\s*\*{2}\s*([^|]*)(?=(?:\|\||$))",
    flags=re.DOTALL
)

# Etiquetas gramaticales a eliminar
grammatical_tags = [
    "m", "f", "pl", "sing", "sust",
    "adj", "adv", "intr", "tr", "prnl", "fr", "interj",
    "coloq", "pop", "rur", "vulg", "despect", "obsol"
]

gender_pattern = r"(?:m(?:\s*\.)?\s*(?:y\s*f(?:\s*\.)?)?)"
utc_pattern_PATTERN = r"U\s*\.\s*t\s*\.\s*c\s*\.\s*(?:s|adj|prnl)\s*\.?"

# Patrón compuesto para eliminar etiquetas gramaticales
drop_tag = re.compile(
    r"(?:\b{mf}\b)|(?:\b(?:{base})(?:\s*\.)?\b)|(?:\b{utc_pattern}\b)".format(
        mf=gender_pattern, base="|".join(grammatical_tags), utc_pattern=utc_pattern_PATTERN
    ),
    flags=re.IGNORECASE
)

# Nombres de regiones colombianas (ordenadas de más largo a más corto para evitar coincidencias parciales)
region_names = [
    "Costa del Pacífico","Costa del Pacifico","Costa Atlántica","Costa Atlantica",
    "Norte de Santander","Valle del Cauca","Llanos Orientales",
    "Costa Pacíf","Costa Pacif","Costa Atl","La Guajira",
    "Cundinamarca","Atlántico","Atlantico","Antioquia","Magdalena","Risaralda",
    "Santander","Putumayo","Caquetá","Caqueta","Casanare","Córdoba","Cordoba",
    "Bolívar","Bolivar","Boyacá","Boyaca","Nariño","Narino","Quindío","Quindio",
    "Amazonas","Caldas","Bogotá","Bogota","Chocó","Choco","Tolima","Cauca","Valle","Huila","Meta","Arauca","Llanos",
    "NStder","Amaz","Stder","Cund","Córd","Cord","Quind","Risar","Magd","Guaj","Tol","Ant","Atl","Bog","Bol","Boy","Cald","Nar"
]

# Mapeo de abreviaturas a nombres completos de regiones
region_full_names = {
    "amaz": "Amazonas",
    "amazonas": "Amazonas",
    "ant": "Antioquia",
    "antioquia": "Antioquia",
    "atl": "Atlántico",
    "atlántico": "Atlántico",
    "atlantico": "Atlántico",
    "bog": "Bogotá",
    "bogotá": "Bogotá",
    "bogota": "Bogotá",
    "bol": "Bolívar",
    "bolívar": "Bolívar",
    "bolivar": "Bolívar",
    "boy": "Boyacá",
    "boyacá": "Boyacá",
    "boyaca": "Boyacá",
    "cald": "Caldas",
    "caldas": "Caldas",
    "chocó": "Chocó",
    "choco": "Chocó",
    "córd": "Córdoba",
    "cord": "Córdoba",
    "córdoba": "Córdoba",
    "cordoba": "Córdoba",
    "costa atl": "Costa Atlántica",
    "costa atlántica": "Costa Atlántica",
    "costa atlantica": "Costa Atlántica",
    "costa pacíf": "Costa del Pacífico",
    "costa pacif": "Costa del Pacífico",
    "costa del pacífico": "Costa del Pacífico",
    "costa del pacifico": "Costa del Pacífico",
    "cund": "Cundinamarca",
    "cundinamarca": "Cundinamarca",
    "guaj": "La Guajira",
    "la guajira": "La Guajira",
    "llanos": "Llanos Orientales",
    "magd": "Magdalena",
    "magdalena": "Magdalena",
    "nar": "Nariño",
    "nariño": "Nariño",
    "narino": "Nariño",
    "nstder": "Norte de Santander",
    "norte de santander": "Norte de Santander",
    "quind": "Quindío",
    "quindío": "Quindío",
    "quindio": "Quindío",
    "risar": "Risaralda",
    "risaralda": "Risaralda",
    "stder": "Santander",
    "santander": "Santander",
    "tol": "Tolima",
    "tolima": "Tolima",
    "valle": "Valle del Cauca",
    "valle del cauca": "Valle del Cauca",
    "cauca": "Cauca",
    "huila": "Huila",
    "meta": "Meta",
    "arauca": "Arauca",
    "casanare": "Casanare",
    "putumayo": "Putumayo",
    "caquetá": "Caquetá",
    "caqueta": "Caquetá"
}

region_pattern_union = "|".join(map(re.escape, region_names))

# Detecta bloques de regiones SOLO en cursiva (entre asteriscos)
# Previene eliminar texto normal como "nar" dentro de "nariz"
italic_region_block = re.compile(
    r"\*\s*(?:(?:{REG})(?:\.|\b))(?:\s*[,you]\s*(?:(?:{REG})(?:\.|\b)))*\s*\*".format(REG=region_pattern_union),
    flags=re.IGNORECASE
)

# Patrones para detectar citas bibliográficas
citation_w_paren_re = re.compile(
    r'(?:-\s*)?\(\s*(?:Carrasquilla|Le[oó]n Rey)\s*,\s*[IVXLCDM]+\s*(?:,\s*(?:copla\s*)?\d+)?\s*"?\s*\)?',
    flags=re.IGNORECASE
)

citation_w_quote_re = re.compile(
    r'(?:-\s*)?[\"""]\s*Le[oó]n Rey\s*,\s*[IVXLCDM]+\s*[\"""]',
    flags=re.IGNORECASE
)

# ============================================================================
# FUNCIONES DE LIMPIEZA DE METADATOS Y EXTRACCIÓN DE REGIONES
# ============================================================================

def remove_citations(text: str) -> str:
    """
    Elimina citas de obras/autor incrustadas en el texto con los formatos detectados.

    Borra patrones como:
    - (Carrasquilla, I, 150")
    - (Carrasquilla, II, 291")
    - (León Rey, II, copla 3932")
    - "León Rey, I"  /  "León Rey, I"

    Además, corrige espacios dobles y comas sobrantes antes de signos.
    """
    if not text:
        return ""
    out = citation_w_paren_re.sub("", text)
    out = citation_w_quote_re.sub("", out)
    out = re.sub(r"\s*,\s*(?=[.,;:])", "", out)
    out = re.sub(r"\s{2,}", " ", out)
    return norm_spaces(out)

def extract_regions_from_text(text: str) -> list:
    """
    Extrae las regiones que están EN CURSIVA (*...*) del texto.
    Retorna una lista de nombres completos de regiones.
    """
    if not text:
        return []
    
    regions_found = []
    for match in italic_region_block.finditer(text):
        block = match.group(0)
        block_clean = block.strip("*").strip()
        # Separar por comas o conjunciones (y, o, u)
        parts = re.split(r'\s*,\s*|\s+(?:y|o|u)\s+', block_clean, flags=re.IGNORECASE)
        for part in parts:
            part = part.strip().rstrip(".").strip()
            if not part:
                continue
            part_lower = part.lower()
            full_name = region_full_names.get(part_lower, None)
            # Intentar sin acentos si no se encuentra
            if full_name is None:
                part_no_accent = part_lower.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
                full_name = region_full_names.get(part_no_accent, None)
            
            # Usar valor original con capitalización si no se encuentra en el diccionario
            if full_name is None:
                full_name = part.title()
            
            if full_name and full_name not in regions_found:
                regions_found.append(full_name)
    
    return regions_found

def clean_regional_blocks_and_grammar_tags(text: str) -> str:
    """
    Limpia del texto:
    1) Bloques de regiones SOLO cuando están en cursiva *...* 
       (p. ej., *Cund., Boy., Nar.*; *Ant., Cald., Valle.*).
    2) Abreviaturas/etiquetas gramaticales con límites de palabra
       (m., f., adj., intr., tr., prnl., fr., interj., coloq., etc., y "U. t. c. ...").
    3) Citas bibliográficas mediante `strip_bibliographic_citations`.
    4) Puntuación/espacios sobrantes tras las limpiezas.

    - No afecta texto normal: no elimina "nar" dentro de "nariz", p ej., 
      porque los bloques regionales se borran solo si están en *cursiva*.
    """
    if not text:
        return ""
    out = text

    out = remove_citations(out)

    # Eliminación iterativa de bloques regionales en cursiva
    while True:
        new_out = italic_region_block.sub("", out)
        if new_out == out:
            break
        out = new_out

    out = drop_tag.sub("", out)

    # Limpieza de puntuación redundante y normalización de espacios
    out = re.sub(r"\s*,\s*(?=[.,;:])", "", out)
    out = re.sub(r"\s*[.,;:—–-]\s*(?=[.,;:—–-])", " ", out)
    out = re.sub(r"\s{2,}", " ", out)
    out = norm_spaces(strip_punct(out))
    return out

# ============================================================================
# DETECCIÓN Y PROCESAMIENTO DE ACEPCIONES ENUMERADAS
# ============================================================================

bold_number_pattern = re.compile(r"\*\*\s*(\d+)\.\s*\*\*")
plain_number_pattern = re.compile(r"(?<!\d)(\d{1,2})\.\s")

def count_enumerations(full_text_after_head: str) -> int:
    """
    Cuenta cuántas definiciones enumeradas (2., 3., etc.) aparecen
    en el texto de un artículo después del lema.

    Se usa para duplicar las filas en el DataFrame por cada acepción enumerada.

    Ejemplos:
   - count_numbered_definitions("**2.** Segunda acepción. **3.** Tercera acepción.") -> 2
    - count_numbered_definitions("Definición simple sin numeración.") -> 0
    """
    count = 0
    for g in bold_number_pattern.findall(full_text_after_head):
        try:
            if int(g) >= 2: count += 1
        except: pass
    for g in plain_number_pattern.findall(full_text_after_head):
        try:
            if int(g) >= 2: count += 1
        except: pass
    return count

enumeration_marker_pattern = re.compile(r"(?:\*\*\s*(\d+)\.\s*\*\*|(?<!\d)(\d{1,2})\.\s)")

def extract_enumerated_senses_from_first_line(first_line: str, head_end_pos: int):
    """
    Extrae y separa las definiciones enumeradas (1., 2., 3., …) de la primera línea del artículo.

    Separa el texto en segmentos para cada acepción numerada, y de cada segmento
    extrae el significado y el ejemplo usando `extract_meaning_example_from_tail`.

    Parámetros:

    first_line : str
        Primera línea completa del artículo (incluyendo el lema y las definiciones).
    head_end_pos : int
        Índice en el que termina el lema en la línea.

    Retorna:

    list of tuple(str, str)
        Lista de tuplas (significado, ejemplo) para cada acepción.

    Ejemplos:
    
     - first_line = "- **carramán** m. coloq. Vehículo viejo. **2.** Persona vieja, acabada."
     -> split_enumerated_definitions_from_line(first_line, 11)
    [('Vehículo viejo.', 'Persona vieja, acabada.')]

     - first_line = "- **carreta** f. Carrete para hilos. **2.** Carretilla para materiales. **3.** Charla trivial."
     -> split_enumerated_definitions_from_line(first_line, 10)
    [('Carrete para hilos.', ''), ('Carretilla para materiales.', ''), ('Charla trivial.', '')]
    """
    tail_raw = (first_line or "")[head_end_pos:]
    tail = tail_raw
    cuts = []
    for m in enumeration_marker_pattern.finditer(tail):
        num = m.group(1) or m.group(2)
        try:
            if int(num) >= 2:
                cuts.append((m.start(), m.end()))
        except:
            pass
    segments = []
    if not cuts:
        segments = [tail]
    else:
        start = 0
        for (s, e) in cuts:
            segments.append(tail[start:s])
            start = e
        segments.append(tail[start:])
    senses = []
    for seg in segments:
        sig, ej, regions = extract_meaning_example_from_tail(seg)
        senses.append((sig, ej, regions))
    return senses

# ============================================================================
# UTILIDADES PARA EXTRACCIÓN DE COMPONENTES DEL LEMA
# ============================================================================

def extract_base_from_head(head_bold_content: str) -> str:
    """
    Extrae la parte base del lema que aparece ANTES del primer '||'.

    Se utiliza para normalizar la palabra principal,
    removiendo puntuación, cursivas y sufijos de género.

    Ejemplos:
    
    - extract_base_from_head("carrera || ~ de encostalados")
     ->'carrera'

    - extract_base_from_head("carranchil (carranchín)")
     ->'carranchil'
    """
    left = head_bold_content.split("||", 1)[0] if "||" in head_bold_content else head_bold_content
    base = strip_punct(clean_curs(norm_spaces(left)))
    base = remove_gender_suffix(base)
    base = strip_punct(base)
    return norm_spaces(base)


def check_meaning_before_pipes(first_line: str, head_end_pos: int) -> bool:
    """
    Verifica si hay contenido significativo (significado) ANTES del primer '||'
    en el texto que sigue al lema.

    Esto permite detectar si el lema principal tiene una definición propia,
    antes de listar ocurrencias derivadas con '||'.

    Ejemplo:
    
    - check_meaning_before_pipes("- **carraca** f. Mandíbula. || echar ~.", 11)
     -> True  # Tiene definición antes de '||'

    - check_meaning_before_pipes("- **carrera || ~ de encostalados** Competencia...", 7)
     -> False # No hay definición antes de '||'
    """
    tail = first_line[head_end_pos:]
    before = tail.split("||", 1)[0]
    clean = clean_regional_blocks_and_grammar_tags(before)
    clean = strip_punct(norm_spaces(clean))
    return bool(clean)

parenthetical_variants_pattern = re.compile(r"\(([^)]+)\)")

def extract_parenthetical_variants(head_bold_content: str) -> list:
    """
    Extrae las variantes indicadas entre paréntesis en el lema principal.

    Ignora los casos en los que el paréntesis solo indica plural opcional (s/es).

    Ejemplos:
    
    - extract_variants_from_parentheses("carranchil (carranchín)")
      -> ['carranchín']

    - extract_variants_from_parentheses("carriel(es)")
      ->  # Ignora plural opcional
    """
    left = head_bold_content.split("||", 1)[0] if "||" in head_bold_content else head_bold_content
    m = parenthetical_variants_pattern.search(left)
    if not m:
        return []
    content = m.group(1)
    if re.fullmatch(r"(?:s|es)", content.strip(), flags=re.IGNORECASE):
        return []
    parts = re.split(r"\s*(?:,|/|;|\bo\b|\bu\b|\by\b)\s*", content, flags=re.IGNORECASE)
    variants = []
    for raw in parts:
        v = norm_spaces(raw)
        if not v: continue
        if re.fullmatch(r"(?:s|es)", v, flags=re.IGNORECASE): continue
        v = remove_gender_suffix(v)
        v = strip_punct(v)
        v = remove_trailing_pipes(v)
        v = norm_spaces(v)
        if v: variants.append(nfc(v))
    return variants

# ============================================================================
# EXTRACCIÓN DE SIGNIFICADO Y EJEMPLO
# ============================================================================

first_italic_pattern = re.compile(r"\*(.*?)\*")

def extract_meaning_example_after_head(first_line: str, head_end_pos: int):
    """
    Extrae el significado y el ejemplo de la PRIMERA LÍNEA de un artículo,
    considerando el texto que sigue al lema.

    Busca la primera frase en cursiva como el ejemplo.
    El resto antes de la cursiva se considera el significado.

    Ejemplo:
    
    - extract_meaning_and_example_from_headline( "- **carraca** f. Mandíbula del hombre. *Había perdido la carraca.*", 11)
      -> ('Mandíbula del hombre.', 'Había perdido la carraca.')
    """
    tail_raw = (first_line or "")[head_end_pos:]
    
    # Extraer regiones ANTES de limpiar metadatos
    regions = extract_regions_from_text(tail_raw)
    
    tail = clean_regional_blocks_and_grammar_tags(tail_raw)
    m = first_italic_pattern.search(tail)
    if m:
        significado = norm_spaces(strip_punct(tail[:m.start()]))
        ejemplo = norm_spaces(strip_punct(m.group(1)))
    else:
        significado = norm_spaces(strip_punct(tail))
        ejemplo = ""
    significado = clean_regional_blocks_and_grammar_tags(significado)
    ejemplo = clean_regional_blocks_and_grammar_tags(ejemplo)
    return significado, ejemplo, regions

def extract_meaning_example_from_tail(tail_raw: str):
    """
    Extrae el significado y el ejemplo desde el resto de texto de un artículo
    (sin el lema).

    Busca el primer texto en cursiva como ejemplo. Lo anterior es el significado.

    Ejemplo:

    - extract_meaning_and_example_from_tail( "Mandíbula del hombre. *Había perdido la carraca.*")
      -> ('Mandíbula del hombre.', 'Había perdido la carraca.')
    """
    # Extraer regiones ANTES de limpiar metadatos
    regions = extract_regions_from_text(tail_raw or "")
    
    tail = clean_regional_blocks_and_grammar_tags(tail_raw or "")
    m = first_italic_pattern.search(tail)
    if m:
        significado = norm_spaces(strip_punct(tail[:m.start()]))
        ejemplo = norm_spaces(strip_punct(m.group(1)))
    else:
        significado = norm_spaces(strip_punct(tail))
        ejemplo = ""
    significado = clean_regional_blocks_and_grammar_tags(significado)
    ejemplo = clean_regional_blocks_and_grammar_tags(ejemplo)
    return significado, ejemplo, regions

# ============================================================================
# SEGMENTACIÓN DEL ARCHIVO EN ARTÍCULOS
# ============================================================================

article_start_regex = re.compile(r"^\s*-?\s*\*\*")
articles, current = [], []
with open(path_source, "r", encoding="utf-8") as f:
    for raw in f:
        line = nfc(raw.rstrip("\n"))
        if article_start_regex.match(line):
            if current:
                articles.append("\n".join(current))
                current = []
        current.append(line) 
    if current:
        articles.append("\n".join(current))

# ============================================================================
# REESCRITURA DE REFERENCIAS "Véase **X**"
# ============================================================================

see_also_pattern = re.compile(r"\bVéase\b\s*\*{2}\s*([^*]+?)\s*\*{2}", flags=re.IGNORECASE)

def _first_line_tail_after_head(first_line: str) -> str:
    """
    Devuelve el texto que sigue inmediatamente después del bloque de cabecera en **negrita**
    dentro de la PRIMERA LÍNEA de un artículo.

    Se asume que la cabecera ya cumple el formato "- **lema** ...".
    Si no hay cabecera detectada, devuelve cadena vacía.

    Ejemplos:
    
    - get_tail_after_head_from_first_line("- **carraca** f. Mandíbula. *Ejemplo*")
      -> " f. Mandíbula. *Ejemplo*"

    - get_tail_after_head_from_first_line("- **carreta || echar ~** fr. coloq. Hablar cosas triviales.")
       -> " fr. coloq. Hablar cosas triviales."
    """
    m = re_bold_head.search(first_line or "")
    if not m:
        return ""
    return (first_line or "")[m.end():]


In [6]:
# ============================================================================
# CONSTRUCCIÓN DE ÍNDICE DE LEMAS Y RESOLUCIÓN DE REFERENCIAS CRUZADAS
# ============================================================================

# Construcción de índice: mapea cada lema normalizado al texto que sigue
# a su encabezado en negrita, necesario para resolver referencias "Véase **X**"
lemma_to_tail = {}
for art in articles:
    fl = art.split("\n", 1)[0]
    head = extract_bold_head_general(fl)
    if not head:
        continue
    lema_norm = normalize_headword_content(head)
    # Almacenar solo la primera ocurrencia de cada lema
    if lema_norm and lema_norm not in lemma_to_tail:
        lemma_to_tail[lema_norm] = _first_line_tail_after_head(fl)

# Resolución de referencias cruzadas "Véase **X**"
# Reemplaza artículos con "Véase **palabra**" por el contenido completo de la palabra referenciada
rewritten_articles = []
for art in articles:
    parts = art.split("\n", 1)
    fl = parts[0]
    rest = parts[1] if len(parts) > 1 else ""
    mh = re_bold_head.search(fl)
    if not mh:
        rewritten_articles.append(art)
        continue

    head_end = mh.end()
    tail = fl[head_end:]

    # Detectar patrón "Véase **lema_referenciado**"
    msee = see_also_pattern.search(tail)
    if not msee:
        rewritten_articles.append(art)
        continue

    # Extraer y normalizar el lema referenciado
    target_raw = msee.group(1)
    target_norm = normalize_headword_content(target_raw)
    target_tail = lemma_to_tail.get(target_norm, "")

    # Si no se encuentra la referencia, mantener el artículo original
    if not target_tail:
        rewritten_articles.append(art)
        continue

    # Reemplazar "Véase **X**" con la definición completa de X
    new_first_line = fl[:head_end] + " " + target_tail.strip()
    new_art = new_first_line if not rest else (new_first_line + "\n" + rest)
    rewritten_articles.append(new_art)

# Actualizar lista de artículos con referencias resueltas
articles = rewritten_articles


## Lógica de Extracción: Requisitos 1 y 2

El pipeline clasifica cada artículo según su estructura para determinar cómo extraer significado y ejemplo:

**Requisito 1 (Artículo Simple):**
- Se aplica cuando: sin pipes (||), sin numeraciones (2., 3., ...)
- Acción: significado y ejemplo de la primera línea → asignados al lema base y variantes

**Requisito 2 (Artículo con Derivados):**
- Se aplica cuando: sin numeraciones, pero con pipes (||) en lema o como ocurrencias externas
- Acción: significado y ejemplo de la primera línea → asignados a cada frase derivada (|| ~)
- Caso especial: si el lema tiene significado antes del primer ||, también genera fila para el lema base

**Artículos Enumerados:**
- Se aplica cuando: contiene numeraciones (2., 3., ...)
- Acción: ignora Req. 1 y 2, procesa cada acepción enumerada independientemente

Esta clasificación evita lógica innecesariamente compleja en artículos simples.

In [7]:
# ============================================================================
# PIPELINE PRINCIPAL: TRANSFORMACIÓN DE ARTÍCULOS A FILAS DEL DATASET
# ============================================================================

# Recorre cada artículo preprocesado y construye filas (palabra, significado, ejemplo, región)
# para el DataFrame final. Aplica lógica diferenciada según estructura del artículo:
# - Artículos simples (Req. 1): una entrada directa por lema
# - Artículos con derivados (Req. 2): múltiples entradas expandiendo pipes y tildes
# - Artículos enumerados: una entrada por cada acepción numerada (2., 3., ...)
#
# Operaciones aplicadas a todos los artículos:
# - Extracción de regiones (ANTES de eliminar metadatos en cursiva)
# - Eliminación de etiquetas gramaticales (m., f., adj., etc.)
# - Eliminación de citas bibliográficas
# - Normalización Unicode (NFC) y limpieza de puntuación/espacios

rows = []

for art in articles:
    first_line = art.split("\n", 1)[0]

    # Extracción y validación del lema en negrita
    head_content = extract_bold_head_general(first_line)
    if not head_content:
        continue

    # Normalización del lema (resuelve pipes/tilde, elimina sufijos, limpia puntuación)
    palabra_head = normalize_headword_content(head_content)
    if not palabra_head:
        continue

    # Localización del fin del encabezado para segmentar el texto
    m_head = re_bold_head.search(first_line)
    head_end = m_head.end() if m_head else 0

    # Texto completo después del lema (para análisis de estructura)
    after_head_all = art[art.find(first_line) + head_end:]

    # Análisis de la estructura del artículo
    has_external_pipes = ("||" in after_head_all)
    head_has_meaning = check_meaning_before_pipes(first_line, head_end) if m_head else False
    n_extra_defs = count_enumerations(after_head_all)
    n_defs = 1 + n_extra_defs
    variants = extract_parenthetical_variants(head_content)

    # Clasificación según Requisitos 1 y 2
    head_has_pipes = ("||" in head_content)
    is_req1_simple = (not has_external_pipes) and (n_extra_defs == 0) and (not head_has_pipes)
    is_req2_simple = (n_extra_defs == 0) and (head_has_pipes or has_external_pipes)

    # Extracción de significado/ejemplo/regiones según clasificación (Req. 1 o Req. 2)
    sig_req1 = ej_req1 = ""
    regions_req1 = []
    sig_req2 = ej_req2 = ""
    regions_req2 = []
    if (m_head and is_req1_simple) or (m_head and is_req2_simple):
        sig, ej, regions = extract_meaning_example_after_head(first_line, head_end)
        if is_req1_simple:
            sig_req1, ej_req1, regions_req1 = sig, ej, regions
        if is_req2_simple:
            sig_req2, ej_req2, regions_req2 = sig, ej, regions

    # Procesamiento de acepciones enumeradas
    enumerated_senses = []
    if n_extra_defs > 0:
        enumerated_senses = extract_enumerated_senses_from_first_line(first_line, head_end)
        # Relleno de segmentos faltantes si la detección fue incompleta
        if len(enumerated_senses) < n_defs:
            enumerated_senses += [("", "", [])] * (n_defs - len(enumerated_senses))

    # Creación de filas para lema base y variantes
    # Excepción: si hay pipes externos sin significado propio, solo se procesan derivados
    if not (has_external_pipes and not head_has_meaning):
        for i in range(n_defs):
            if n_extra_defs > 0:
                sig_i, ej_i, regions_i = enumerated_senses[i]
            else:
                sig_i = (sig_req1 if is_req1_simple else (sig_req2 if is_req2_simple else ""))
                ej_i  = (ej_req1  if is_req1_simple else (ej_req2  if is_req2_simple else ""))
                regions_i = (regions_req1 if is_req1_simple else (regions_req2 if is_req2_simple else []))
            rows.append({
                "palabra": palabra_head,
                "significado": sig_i,
                "ejemplo": ej_i,
                "región": ", ".join(regions_i) if regions_i else ""
            })
        
        # Filas para variantes del lema
        for var in variants:
            for i in range(n_defs):
                if n_extra_defs > 0:
                    sig_i, ej_i, regions_i = enumerated_senses[i]
                else:
                    sig_i = (sig_req1 if is_req1_simple else (sig_req2 if is_req2_simple else ""))
                    ej_i  = (ej_req1  if is_req1_simple else (ej_req2  if is_req2_simple else ""))
                    regions_i = (regions_req1 if is_req1_simple else (regions_req2 if is_req2_simple else []))
                rows.append({
                    "palabra": var,
                    "significado": sig_i,
                    "ejemplo": ej_i,
                    "región": ", ".join(regions_i) if regions_i else ""
                })

    # Creación de filas para ocurrencias externas (derivados con || **...~...**)
    if has_external_pipes:
        base_raw = extract_base_from_head(head_content)

        for bold_phrase, local_tail in external_occurrence_pattern.findall(after_head_all):
            # Expansión de tilde y limpieza
            phrase = replace_tilde_with_base(bold_phrase, base_raw)
            phrase = clean_curs(phrase)
            phrase = strip_punct(remove_trailing_pipes(norm_spaces(phrase))).rstrip(".")
            if not phrase:
                continue

            # Extracción de significado/ejemplo/regiones si no hay enumeraciones
            sig_loc = ej_loc = ""
            regions_loc = []
            if n_extra_defs == 0:
                sig_loc, ej_loc, regions_loc = extract_meaning_example_from_tail(local_tail)

            rows.append({
                "palabra": phrase,
                "significado": sig_loc,
                "ejemplo": ej_loc,
                "región": ", ".join(regions_loc) if regions_loc else ""
            })

# Construcción del DataFrame final
df = pd.DataFrame(rows).reset_index(drop=True)


In [8]:
# ============================================================================
# LIMPIEZA FINAL Y EXPORTACIÓN A JSON
# ============================================================================

out_file_json = "../BDC.json"

# Limpieza y normalización de columnas de texto
for col in ['palabra', 'significado', 'ejemplo']:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].str.replace(r'^[\'"]|[\'"]$', '', regex=True)
    df[col] = df[col].str.lower()

# Normalización de columna de regiones (mantiene capitalización original)
df['región'] = df['región'].astype(str).str.strip()

# Ordenamiento alfabético por palabra
df = df.sort_values(by='palabra', ascending=True).reset_index(drop=True)

# Exportación a JSON
data_json = df.to_dict(orient='records')

with open(out_file_json, 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=2)

print(f"\nJSON guardado en: {out_file_json}")
print(f"Total de entradas: {len(df)}")



JSON guardado en: ../BDC.json
Total de entradas: 2693
